In [128]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import datetime
import pandas as pd
from datetime import datetime

In [60]:
data = {"dates" : [],
        "values" : []
        }

In [61]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    counter = 0
    iter = 0
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass


        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )          
             
        counter+=1

        if counter == 5:
            counter = 0
            iter += 1
            avg_y = (landmarks[15].y+landmarks[16].y+landmarks[17].y+landmarks[18].y+
                        landmarks[19].y+landmarks[20].y+landmarks[21].y+landmarks[22].y)/8

            data["values"].append(avg_y)
            data["dates"].append(datetime.datetime.today().strftime("%d-%m-%y %H:%M:%S"))
            if iter == 180:
                break
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [103]:
df_raw = pd.DataFrame(data = data)

In [64]:
df_raw.to_csv("y_axis_bench.csv" , index = False)

In [118]:
df = df_raw[(df_raw['values'] > df_raw['values'].quantile(q=0.75)) |
            (df_raw['values'] < df_raw['values'].quantile(q=0.25)) ]

<AxesSubplot:xlabel='dates'>

In [257]:
%matplotlib qt
df.plot(kind='line', y = "values", x = "dates", style='-o')
df_raw.plot(kind='line', y = "values", x = "dates", style='-o')

<AxesSubplot:xlabel='dates'>

In [266]:

import math

df['Speed'] = df['values'].diff() / df['values'].count()
times = pd.Series(datetime.strptime(df['dates'][0],"%d-%m-%y %H:%M:%S"))
df['Rest_Time'] = times.diff()
df['Intensity'] = (df['Speed'] + df['values']) / 2
df['Fatigue'] = df['Intensity'].diff()

s = df['Speed'].values

results = []
counter = 0
for i in range(len(s)-1):
    s[i] *= 100000
    if math.isnan(s[i]):
        s[i] = 0
    if abs(s[i]) < 60:
        counter+=1
    else:
        if abs(s[i-1]) < 60 :
            results.append(counter)
            counter=0

print(results)

[1, 1, 2, 5, 2, 1, 1, 1, 1, 5, 2, 4, 2, 4, 3, 1, 3, 4, 4, 3, 3, 4]


C:\Users\mosac\AppData\Local\Temp\ipykernel_15096\1774298178.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Speed'] = df['values'].diff() / df['values'].count()
C:\Users\mosac\AppData\Local\Temp\ipykernel_15096\1774298178.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rest_Time'] = times.diff()
C:\Users\mosac\AppData\Local\Temp\ipykernel_15096\1774298178.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [264]:
results

[1, 1, 2, 5, 2, 1, 1, 1, 1, 5, 2, 4, 2, 4, 3, 1, 3, 4, 4, 3, 3, 4]

In [268]:
df

,dates,values,Speed,Rest_Time,Intensity,Fatigue
0,12-02-23 10:34:42,0.692126,0.000000,NaT,NaN,NaN
2,12-02-23 10:34:42,0.588290,-115.373848,NaT,0.293568,NaN
3,12-02-23 10:34:43,0.577750,-11.710767,NaT,0.288817,-0.004752
5,12-02-23 10:34:43,0.442471,-150.310798,NaT,0.220484,-0.068333
6,12-02-23 10:34:44,0.402356,-44.571216,NaT,0.200955,-0.019528
...,...,...,...,...,...,...
175,12-02-23 10:35:33,0.437713,2.748352,NaT,0.218870,0.001259
176,12-02-23 10:35:34,0.434830,-3.203327,NaT,0.217399,-0.001471
177,12-02-23 10:35:34,0.435694,0.959105,NaT,0.217852,0.000452
178,12-02-23 10:35:34,0.439504,4.234137,NaT,0.219773,0.001922
